# DSR Batch 28 - Team 3 Rossmann Mini-competition by Eero Olli, Jose Soldado, Hazel Wat

In [417]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import category_encoders as ce

#Profilibg to check data
from pandas_profiling import ProfileReport

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import power_transform
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from datetime import datetime 
from datetime import date 

In [418]:
!python --version

Python 3.8.12


In [419]:
# reading two csv files
train = pd.read_csv('data/train.csv',dtype={'Store':'string', 'StateHoliday':'string'})
store = pd.read_csv('data/store.csv',dtype={'Store':'string', 'StoreType':'string', 'Assortment':'string'})
  
# using merge function by setting how='inner'
total = pd.merge(train, store, on='Store', how='inner')

In [420]:
total

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,1115,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1,2013-01-02,1115,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
2,2013-01-03,1115,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
3,2013-01-04,1115,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
4,2013-01-05,1115,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618468,2014-06-26,127,4.0,3335.0,480.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618469,2014-06-27,127,5.0,4724.0,591.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618470,2014-06-28,127,6.0,5398.0,716.0,1.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
618471,2014-06-29,127,7.0,0.0,0.0,0.0,0.0,0,0.0,d,a,1350.0,12.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [421]:
#Train test split processing
X = total.drop(columns='Sales')
y = total.loc[:,'Sales']

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [423]:
#Merge train data (X and y) to process them together
df_train = pd.concat([X_train,y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [424]:
#prof1 = ProfileReport(df_train)

In [425]:
#prof1.to_file(output_file='prof1.html')

In [426]:
#Drop na values in sales

df_train = df_train.dropna(subset =['Sales'])
df_test = df_test.dropna(subset =['Sales'])

#Drop columns Customers and PromoInterval

df_train = df_train.drop(columns=['Customers','PromoInterval'])
df_test = df_test.drop(columns=['Customers','PromoInterval'])

## Data Preparation

In [427]:
# Competitor distance : missing values
df_train['CompetitionDistance'] = df_train.loc[:,'CompetitionDistance'].fillna(df_train['CompetitionDistance'].median())                                                                            
df_test['CompetitionDistance']= df_test.loc[:,'CompetitionDistance'].fillna(df_test['CompetitionDistance'].median())

In [428]:
# Check how many stores has missing values on Open column
df_train['Open'].isna().sum()

14391

In [429]:
#Open: Referring to Sales to fill NA values

df_train.loc[df_train['Sales']> 0, 'Open']=1
df_train.loc[df_train['Sales']== 0, 'Open']=0

df_test.loc[df_test['Sales']> 0, 'Open']=1
df_test.loc[df_test['Sales']== 0, 'Open']=0

In [430]:
df_train['Open'].value_counts()

1.0    397880
0.0     82156
Name: Open, dtype: int64

In [431]:
#SatateHoloiday: fillna with 0 and get a string
df_train['StateHoliday'] = df_train.loc[:,'StateHoliday'].fillna('0')
df_train['StateHoliday'] = df_train['StateHoliday'].replace(0.0, '0')

df_test['StateHoliday'] = df_test.loc[:,'StateHoliday'].fillna('0')
df_test['StateHoliday'] = df_test['StateHoliday'].replace(0.0, '0')

In [432]:
df_train.StateHoliday.unique()

<StringArray>
['0', 'a', 'c', 'b']
Length: 4, dtype: string

In [433]:
df_train['SchoolHoliday'].isnull().value_counts()

False    465424
True      14612
Name: SchoolHoliday, dtype: int64

In [434]:
df_train["SchoolHoliday"].isna().value_counts()

False    465424
True      14612
Name: SchoolHoliday, dtype: int64

In [435]:
#Schoolholiday fillna using 0
df_train['SchoolHoliday'] = df_train['SchoolHoliday'].fillna(0)
df_test['SchoolHoliday'] = df_test['SchoolHoliday'].fillna(0)

In [436]:
#Tranform Data into datatime pd to get the day, week and year
df_train['Sales_date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = pd.DatetimeIndex(df_train['Date']).year
df_train['Month'] = pd.DatetimeIndex(df_train['Date']).month
df_train['Day'] = pd.DatetimeIndex(df_train['Date']).day

df_test['Sales_date'] = pd.to_datetime(df_test['Date'])
df_test['Year'] = pd.DatetimeIndex(df_test['Date']).year
df_test['Month'] = pd.DatetimeIndex(df_test['Date']).month
df_test['Day'] = pd.DatetimeIndex(df_test['Date']).day

In [437]:
df_train.head()

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Sales,Sales_date,Year,Month,Day
290482,2013-11-19,175,2.0,1.0,1.0,0,0.0,c,a,4130.0,NaN,NaN,0,NaN,NaN,5982.0,2013-11-19,2013,11,19
139915,2013-04-13,468,6.0,1.0,0.0,0,0.0,c,c,5260.0,9.0,2012.0,0,NaN,NaN,8256.0,2013-04-13,2013,4,13
318657,2013-07-22,941,1.0,1.0,0.0,0,0.0,a,a,1200.0,12.0,2011.0,1,31.0,2013.0,4714.0,2013-07-22,2013,7,22
361617,2014-01-06,894,1.0,1.0,1.0,0,1.0,a,a,190.0,11.0,2012.0,0,NaN,NaN,13744.0,2014-01-06,2014,1,6
374646,2013-05-24,975,5.0,1.0,0.0,0,0.0,a,c,9630.0,NaN,NaN,1,14.0,2011.0,7308.0,2013-05-24,2013,5,24


In [438]:
df_train['Promo'].isna().value_counts()

False    465545
True      14491
Name: Promo, dtype: int64

In [439]:
#Promo missing values (0)
df_train['Promo'] = df_train.loc[:,'Promo'].fillna(0)
df_test['Promo'] = df_test.loc[:,'Promo'].fillna(0)

In [440]:
df_train['Promo2'].isna().value_counts()

False    480036
Name: Promo2, dtype: int64

In [441]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 618473 entries, 0 to 618472
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       618473 non-null  object 
 1   Store                      618473 non-null  string 
 2   DayOfWeek                  600036 non-null  float64
 3   Sales                      600028 non-null  float64
 4   Customers                  599957 non-null  float64
 5   Open                       599832 non-null  float64
 6   Promo                      599791 non-null  float64
 7   StateHoliday               599873 non-null  string 
 8   SchoolHoliday              599686 non-null  float64
 9   StoreType                  618473 non-null  string 
 10  Assortment                 618473 non-null  string 
 11  CompetitionDistance        616838 non-null  float64
 12  CompetitionOpenSinceMonth  422043 non-null  float64
 13  CompetitionOpenSinceYear   42

In [442]:
def create_a_timeseries_dummy_from_weeks(df, var_week, var_year, date_new, var_dummy):
    #Create a date out of the Promo2 variables so that we can create a time-series variable for Promo2
    df['temp_date'] = df[var_year] * 1000 + df[var_week]  * 10 + 0
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%W%w')
    ## Create an array to Group the data by group. and make dummy Store by Store
    for Store, grouped in df.groupby('Store'):
      if [df['Sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])
    df.head()

    
def create_a_timeseries_dummy_from_months(df, var_month, var_year, date_new, var_dummy):
    #Create a date out of the Competition variables so that we can create a time-series variable for Competitioin
    df['temp_date'] = df[var_year] * 1000 + df[var_month]  * 10 + 0
    ## Create an array to Group the data by group. and make dummy Store by Store
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%m%w')
    for Store, grouped in df.groupby('Store'):
      if [df['Sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])    
    df.head()

In [443]:
create_a_timeseries_dummy_from_weeks(df=df_train        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

create_a_timeseries_dummy_from_weeks(df=df_test        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

In [444]:
create_a_timeseries_dummy_from_months(df=df_train                  ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )

create_a_timeseries_dummy_from_months(df=df_test                 ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )


In [445]:
df_train

,Date,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Sales,Sales_date,Year,Month,Day,temp_date,Promo2_start_date,Promo2_yes,Competition_start_date,Competition_yes
290482,2013-11-19,175,2.0,1.0,1.0,0,0.0,c,a,4130.0,...,5982.0,2013-11-19,2013,11,19,NaN,NaT,1,NaT,1
139915,2013-04-13,468,6.0,1.0,0.0,0,0.0,c,c,5260.0,...,8256.0,2013-04-13,2013,4,13,2012090.0,NaT,1,2012-09-01,1
318657,2013-07-22,941,1.0,1.0,0.0,0,0.0,a,a,1200.0,...,4714.0,2013-07-22,2013,7,22,2011120.0,2013-08-11,1,2011-12-01,1
361617,2014-01-06,894,1.0,1.0,1.0,0,1.0,a,a,190.0,...,13744.0,2014-01-06,2014,1,6,2012110.0,NaT,1,2012-11-01,1
374646,2013-05-24,975,5.0,1.0,0.0,0,0.0,a,c,9630.0,...,7308.0,2013-05-24,2013,5,24,NaN,2011-04-10,1,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,2014-01-13,536,1.0,1.0,0.0,0,0.0,a,c,4700.0,...,6523.0,2014-01-13,2014,1,13,2002090.0,2013-08-11,1,2002-09-01,1
259178,2014-05-09,256,5.0,1.0,0.0,0,0.0,a,c,80.0,...,10356.0,2014-05-09,2014,5,9,2005090.0,2014-03-16,1,2005-09-01,1
365838,2013-06-25,873,2.0,1.0,0.0,0,0.0,a,a,2040.0,...,2763.0,2013-06-25,2013,6,25,2008110.0,NaT,1,2008-11-01,1
131932,2014-04-05,483,6.0,1.0,0.0,0,0.0,a,c,2310.0,...,1581.0,2014-04-05,2014,4,5,2011090.0,2010-04-04,1,2011-09-01,1


In [446]:
#Drop columns
df_train = df_train.drop(columns=['Promo2_start_date','Competition_start_date','temp_date','Date', 'Promo2', 
                                  'Sales_date','CompetitionOpenSinceYear',
    'Promo2SinceWeek','Promo2SinceYear','CompetitionOpenSinceMonth'])

df_test = df_test.drop(columns=['Promo2_start_date','Competition_start_date','temp_date','Date', 'Promo2',
                                'Sales_date','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear',
                                'CompetitionOpenSinceMonth'])

In [447]:
df_train

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Sales,Year,Month,Day,Promo2_yes,Competition_yes
290482,175,2.0,1.0,1.0,0,0.0,c,a,4130.0,5982.0,2013,11,19,1,1
139915,468,6.0,1.0,0.0,0,0.0,c,c,5260.0,8256.0,2013,4,13,1,1
318657,941,1.0,1.0,0.0,0,0.0,a,a,1200.0,4714.0,2013,7,22,1,1
361617,894,1.0,1.0,1.0,0,1.0,a,a,190.0,13744.0,2014,1,6,1,1
374646,975,5.0,1.0,0.0,0,0.0,a,c,9630.0,7308.0,2013,5,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,536,1.0,1.0,0.0,0,0.0,a,c,4700.0,6523.0,2014,1,13,1,1
259178,256,5.0,1.0,0.0,0,0.0,a,c,80.0,10356.0,2014,5,9,1,1
365838,873,2.0,1.0,0.0,0,0.0,a,a,2040.0,2763.0,2013,6,25,1,1
131932,483,6.0,1.0,0.0,0,0.0,a,c,2310.0,1581.0,2014,4,5,1,1


In [448]:
df_train.DayOfWeek = df_train["DayOfWeek"].fillna(1)
df_test.DayOfWeek = df_test["DayOfWeek"].fillna(1)

### Encodings

In [449]:
# these are already dummies 'Open','Promo', 'SchoolHoliday',,'Promo2' 'DayOfWeek',
ce_one = ce.OneHotEncoder(cols=['StateHoliday','StoreType','Assortment'])  #instantiate the encoder

df_train = ce_one.fit_transform(df_train)
df_test = ce_one.transform(df_test)

In [451]:
df_train.head()

,Store,DayOfWeek,Open,Promo,StateHoliday_1,StateHoliday_2,StateHoliday_3,StateHoliday_4,SchoolHoliday,StoreType_1,...,Assortment_1,Assortment_2,Assortment_3,CompetitionDistance,Sales,Year,Month,Day,Promo2_yes,Competition_yes
290482,175,2.0,1.0,1.0,1,0,0,0,0.0,1,...,1,0,0,4130.0,5982.0,2013,11,19,1,1
139915,468,6.0,1.0,0.0,1,0,0,0,0.0,1,...,0,1,0,5260.0,8256.0,2013,4,13,1,1
318657,941,1.0,1.0,0.0,1,0,0,0,0.0,0,...,1,0,0,1200.0,4714.0,2013,7,22,1,1
361617,894,1.0,1.0,1.0,1,0,0,0,1.0,0,...,1,0,0,190.0,13744.0,2014,1,6,1,1
374646,975,5.0,1.0,0.0,1,0,0,0,0.0,0,...,0,1,0,9630.0,7308.0,2013,5,24,1,1


In [452]:
df_train.isnull().sum()

Store                  0
DayOfWeek              0
Open                   0
Promo                  0
StateHoliday_1         0
StateHoliday_2         0
StateHoliday_3         0
StateHoliday_4         0
SchoolHoliday          0
StoreType_1            0
StoreType_2            0
StoreType_3            0
StoreType_4            0
Assortment_1           0
Assortment_2           0
Assortment_3           0
CompetitionDistance    0
Sales                  0
Year                   0
Month                  0
Day                    0
Promo2_yes             0
Competition_yes        0
dtype: int64

In [453]:
df_train[df_train["Sales"] == 0].index

Int64Index([   166,  45019, 311788, 323992, 514020,  37974, 258518, 532784,
            335527, 246000,
            ...
            208261, 256840, 331236, 500186, 421909, 347449, 214176, 327069,
            278167, 521430],
           dtype='int64', length=82156)

In [454]:
df_train = df_train.drop(df_train[df_train["Sales"] == 0].index)
df_test = df_test.drop(df_test[df_test["Sales"] == 0].index)

In [455]:
df_train.shape

(397880, 23)

In [457]:
#Split data
X_train = df_train.drop(columns=['Sales'])
y_train = np.asarray(df_train.loc[:,'Sales'])

X_test = df_test.drop(columns=['Sales'])
y_test = np.asarray(df_test.loc[:,'Sales'])

In [458]:
X_train.dtypes

Store                   string
DayOfWeek              float64
Open                   float64
Promo                  float64
StateHoliday_1           int64
StateHoliday_2           int64
StateHoliday_3           int64
StateHoliday_4           int64
SchoolHoliday          float64
StoreType_1              int64
StoreType_2              int64
StoreType_3              int64
StoreType_4              int64
Assortment_1             int64
Assortment_2             int64
Assortment_3             int64
CompetitionDistance    float64
Year                     int64
Month                    int64
Day                      int64
Promo2_yes               int64
Competition_yes          int64
dtype: object

In [459]:
# Scale 
scal = StandardScaler()
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)

## Model Development

#### Linear Regression

In [460]:
#Model Linear Regression
regr = LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [461]:
#Prediction on training Data
y_pred_train = regr.predict(X_train)
y_pred_test = regr.predict(X_test)

In [462]:
#Building required metric
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [463]:
train_s = metric(y_pred_train,y_train)
test_s = metric(y_pred_test,y_test)

In [464]:
train_s

52.38602729813417

In [465]:
test_s

51.151081962751725

#### Gradient Boosted Trees

In [466]:
# instanstiate the regressor 
xgbr = xgb.XGBRegressor(n_estimators = 1500, max_depth = 7, learning_rate = 0.15, n_jobs= -1)

#fit the regressor
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [467]:
#predict with xgbr
y_pred_xgbr = xgbr.predict(X_test)
y_pred_xgbr_train = xgbr.predict(X_train)

In [468]:
metric_xgbr_train = metric(y_pred_xgbr_train, y_train)
metric_xgbr_train

13.077925553419341

In [469]:
y_pred_xgbr

array([ 7595.974 ,  5020.732 ,  4776.148 , ...,  6051.627 , 20716.574 ,
        3714.1477], dtype=float32)

In [470]:
#Accuracy
metric_xgbr = metric(y_pred_xgbr,y_test)
metric_xgbr

12.641089691961811

### Grid Search CV

In [ ]:
# Create the parameter grid based on the results of random search
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.1,0.15,0.05],
    'max_depth': [3, 5, 7],
    'n_estimators': [800, 1000, 1500]
}

# Create a based model
xgbr_cv = xgb.XGBRegressor(n_jobs=-1)# Instantiate the grid search model

grid_search = GridSearchCV(estimator = xgbr_cv, param_grid = param_grid,
                          cv = 3)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

In [ ]:
grid_accuracy = evaluate(best_grid, X_test, y_test)

### Plot Graph

In [472]:
# Compute the correlation matrix 
# exclude 'Open' variable
#corr_all = df_randomf.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle
#mask = np.zeros_like(corr_all, dtype = np.bool)
#mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
#f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
#sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "BuPu")      
#plt.show()

IndentationError: unexpected indent (2931055050.py, line 14)